In [1]:
import numpy
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
pops=['CLCM','CLPO','CLRA',
      'CMSC','DBPO','DBRA',
      'DBSK','DBTB','HPPD','HPTB',
      'PDTB','POSK','SCSK','SCTB']

In [3]:
def BestModel(dataset):
    
    '''
    Parameters
    L is the total length of DNA sequence (in bp) that was analyzed in order to obtain the SNP data
    mu is mutation rate
    '''
    mu=10e-9
    L=164229310 #sites analyses by angsd first round
    
    #Read results table
    results=pd.read_table('results/'+dataset+'_demoIM_fits.txt',\
                          names=["ll","s","nu1","nu2","time","m12","m21","theta"])
    
    #Select model with maximum likelihood
    model_sel=results.loc[results.ll.idxmax()]
    
    #Ancestral population size
    Nref=model_sel.theta/(4*mu*L)
    
    #Calculate model parameters
    pop1_Ne_aftersplit=model_sel.s*2*Nref 
    pop2_Ne_aftersplit=(1-model_sel.s)*2*Nref 
    pop1_Ne=model_sel.nu1 * 2*Nref 
    pop2_Ne=model_sel.nu2 * 2*Nref 
    
    print(dataset,results.shape[0])
    
    #Results in list
    results=[dataset,round(model_sel.ll),round(pop1_Ne_aftersplit),round(pop1_Ne),round(pop2_Ne_aftersplit),\
    round(pop2_Ne),round(model_sel.time*2*Nref),round(model_sel.time*2*Nref)*2]
    
    
    
    return results

In [4]:
allresults=[]
for pair in pops:
    allresults.append(BestModel(pair))

CLCM 103
CLPO 100
CLRA 100
CMSC 106
DBPO 110
DBRA 100
DBSK 106
DBTB 108
HPPD 109
HPTB 108
PDTB 101
POSK 100
SCSK 113
SCTB 100


In [5]:
colnames=["pops","loglikelihood","pop1","pop1_growth","pop2","pop2_growth","split_time_gen","split_time_years"]
resultsdf=pd.DataFrame(allresults,columns=colnames)
resultsdf

,pops,loglikelihood,pop1,pop1_growth,pop2,pop2_growth,split_time_gen,split_time_years
0,CLCM,-536946,79094,318615,107688,114511,11024,22048
1,CLPO,-470867,74089,385556,110925,223680,15042,30084
2,CLRA,-242042,105533,424957,54078,415668,17950,35900
3,CMSC,-1041896,122414,104752,72952,66083,17229,34458
4,DBPO,-1156174,126747,170478,92306,125200,8973,17946
5,DBRA,-311191,182946,192739,11669,1044829,2546,5092
6,DBSK,-311863,98868,191775,98868,219930,486,972
7,DBTB,-1468353,115185,183133,118897,88007,15942,31884
8,HPPD,-202484,139930,210381,25358,3849234,16714,33428
9,HPTB,-722806,101314,116500,106797,110365,10694,21388


In [6]:
pop=list(resultsdf["pops"].str[0:2])+list(resultsdf["pops"].str[2:4])
Nes=list(resultsdf["pop1_growth"])+list(resultsdf["pop2_growth"])

In [7]:
modelresults=pd.DataFrame(zip(pop,Nes),columns=["pop","Ne"])
modelConsistency=modelresults.groupby("pop").agg({'Ne': ['count', 'min', 'max']})["Ne"]
modelConsistency["ratio"]=modelConsistency["max"]/modelConsistency["min"]
modelConsistency

,count,min,max,ratio
pop,,,,
CL,3,318615,424957,1.333763
CM,2,104752,114511,1.093163
DB,4,170478,192739,1.130580
HP,2,116500,210381,1.805845
PD,2,3160165,3849234,1.218048
PO,3,125200,223680,1.786581
RA,2,415668,1044829,2.513614
SC,3,66083,128697,1.947505
SK,3,162801,219930,1.350913


In [8]:
resultsdf.to_csv("dadiResultsNoMig.txt",sep="\t",index=False)